In [1]:
# -*- coding: utf-8 -*-
"""
Fetching atlases and annotations
================================

This example demonstrates how to use :mod:`neuromaps.datasets` to fetch
atlases and annotations.
"""

###############################################################################
# Much of the functionality of the ``neuromaps`` toolbox relies on the
# atlases and atlas files provided with it. In many cases these atlases are
# fetched "behind-the-scenes" when you call functions that depend on them, but
# they can be accessed directly.
#
# There is a general purpose :func:`neuromaps.datasets.fetch_atlas`
# function that can fetch any of the atlases provided with ``neuromaps``:

from neuromaps import datasets

fslr = datasets.fetch_atlas(atlas='fslr', density='32k')
print(fslr.keys())

###############################################################################
# The values corresponding to the keys of the atlas dictionary are length-2
# lists containing filepaths to the downloaded data. All surface atlas files
# are provide in gifti format (whereas MNI files are in gzipped nifti format).
#
# You can load them directly with ``nibabel`` to confirm their validity:

import nibabel as nib
lsphere, rsphere = fslr['sphere']
lvert, ltri = nib.load(lsphere).agg_data()
print(lvert.shape, ltri.shape)

###############################################################################
# The other datasets that are provided with ``neuromaps`` are annotations
# (i.e., brain maps!). While we are slowly making more and more of these openly
# available, for now only a subset are accessible to the general public; these
# are returned by default via :func:`datasets.available_annotations`.

annotations = datasets.available_annotations()
print(f'Available annotations: {len(annotations)}')

###############################################################################
# The :func:`~.available_annotations` function accepts a number of keyword
# arguments that you can use to query specific datasets. For example, providing
# the `format='volume`' argument will return only those annotations that
# are, by default, a volumetric image:

volume_annotations = datasets.available_annotations(format='volume')
print(f'Available volumetric annotations: {len(volume_annotations)}')

###############################################################################
# There are a number of keyword arguments we can specify to reduce the scope of
# the annotations returned. Here, `source` specifies where the annotation came
# from (i.e., a dataset from a manuscript or a data repository or toolbox),
# `desc` refers to a brief description of the annotation, `space` clarifies
# which space the annotation is in, and `den` (specific to surface annotations)
# clarifies the density of the surface on which the annotation is defined:

annot = datasets.available_annotations(source='abagen', desc='genepc1',
                                       space='fsaverage', den='10k')
print(annot)

###############################################################################
# Annotations also have tags to help sort them into categories. You can see
# what tags can be used to query annotations with the :func:`~.available_tags`
# functions:

tags = datasets.available_tags()
print(tags)

###############################################################################
# Tags can be used as a keyword argument with :func:`~.available_annotations`.
# You can supply either a single tag or a list of tags. Note that supplying a
# list will only return those annotations that match ALL supplied tags:

fmri_annotations = datasets.available_annotations(tags='fMRI')
print(fmri_annotations)

###############################################################################
# Once we have an annotation that we want we can use the
# :func:`neuromaps.datasets.fetch_annotation` to actually download the
# files. This has a very similar signature to the
# :func:`~.available_annotations` function, accepting almost all the same
# keyword arguments to specify which annotations are desired.
#
# Here, we'll grab the first principal component of gene expression across the
# brain (from the Allen Human Brain Atlas):

abagen = datasets.fetch_annotation(source='abagen', desc='genepc1')
print(abagen)

###############################################################################
# Notice that the returned annotation ``abagen`` is a dictionary. We can subset
# the dictionary with the appropriate key or, if we know that our query is
# going to return only one annotation, also provide the `return_single=True`
# argument to the fetch call:

abagen = datasets.fetch_annotation(source='abagen', desc='genepc1',
                                   return_single=True)
print(abagen)

###############################################################################
# And that's it! This example provided a quick overview on how to fetch the
# various atlases and datasets provided with ``neuromaps``. For more
# information please refer to the :ref:`API reference <ref_datasets>`.

 ...done. (3 seconds, 0 min)
Extracting data from /home/tamires/neuromaps-data/599046a594e0e45c04e90291c2348cbe/fsLR32k.tar.gz..... done.


dict_keys(['midthickness', 'inflated', 'veryinflated', 'sphere', 'medial', 'sulc', 'vaavg'])
(32492, 3) (64980, 3)
Available annotations: 80
Available volumetric annotations: 41
[('abagen', 'genepc1', 'fsaverage', '10k')]
['ASL', 'MEG', 'MRI', 'PET', 'fMRI', 'functional', 'genetics', 'meta-analysis', 'metabolism', 'receptors', 'resteyesopen', 'structural']
[('margulies2016', 'fcgradient01', 'fsLR', '32k'), ('margulies2016', 'fcgradient02', 'fsLR', '32k'), ('margulies2016', 'fcgradient03', 'fsLR', '32k'), ('margulies2016', 'fcgradient04', 'fsLR', '32k'), ('margulies2016', 'fcgradient05', 'fsLR', '32k'), ('margulies2016', 'fcgradient06', 'fsLR', '32k'), ('margulies2016', 'fcgradient07', 'fsLR', '32k'), ('margulies2016', 'fcgradient08', 'fsLR', '32k'), ('margulies2016', 'fcgradient09', 'fsLR', '32k'), ('margulies2016', 'fcgradient10', 'fsLR', '32k'), ('mueller2013', 'intersubjvar', 'fsLR', '164k'), ('neurosynth', 'cogpc1', 'MNI152', '2mm')]


 ...done. (1 seconds, 0 min)


 ...done. (3 seconds, 0 min)


['/home/tamires/neuromaps-data/annotations/abagen/genepc1/fsaverage/source-abagen_desc-genepc1_space-fsaverage_den-10k_hemi-L_feature.func.gii', '/home/tamires/neuromaps-data/annotations/abagen/genepc1/fsaverage/source-abagen_desc-genepc1_space-fsaverage_den-10k_hemi-R_feature.func.gii']
['/home/tamires/neuromaps-data/annotations/abagen/genepc1/fsaverage/source-abagen_desc-genepc1_space-fsaverage_den-10k_hemi-L_feature.func.gii', '/home/tamires/neuromaps-data/annotations/abagen/genepc1/fsaverage/source-abagen_desc-genepc1_space-fsaverage_den-10k_hemi-R_feature.func.gii']


In [2]:
# -*- coding: utf-8 -*-
"""
Using spatial null models
=========================

This example demonstrates how to use spatial null models in
:mod:`neuromaps.nulls` to test the correlation between two brain
annotations.
"""

###############################################################################
# The brain—and most features derived from it—is spatially autocorrelated, and
# therefore when making comparisons between brain features we need to account
# for this spatial autocorrelation.
#
# Enter: spatial null models.
#
# Spatial null models need to be used whenever you're comparing brain maps. In
# order to demonstrate how use them in ``neuromaps`` we need two
# annotations to compare. We'll use the first principal component of cognitive
# terms from NeuroSynth (Yarkoni et al., 2011, Nat Methods) and the first
# principal component of gene expression across the brain (from the Allen Human
# Brain Atlas).
#
# Note that we pass `return_single=True` to
# :func:`neuromaps.datasets.fetch_annotation` so that the returned data are
# a list of filepaths rather than the default dictionary format. (This only
# works since we know that there is only one annotation matching our query; a
# dictionary will always be returned if multiple annotations match our query.)

from neuromaps import datasets
nsynth = datasets.fetch_annotation(source='neurosynth', return_single=True)
genepc = datasets.fetch_annotation(desc='genepc1', return_single=True)
print('Neurosynth: ', nsynth)
print('Gene PC1: ', genepc)

###############################################################################
# These annotations are in different spaces so we first need to resample them
# to the same space. Here, we'll choose to resample them to the 'fsaverage'
# surface with a '10k' resolution (approx 10k vertices per hemisphere). Note
# that the `genepc1` is already in this space so no resampling will be
# performed for those data. (We could alternatively specify 'transform_to_trg'
# for the `resampling` parameter and achieve the same outcome.)
#
# The data returned will always be pre-loaded nibabel image instances:

from neuromaps import resampling
nsynth, genepc = resampling.resample_images(src=nsynth, trg=genepc,
                                            src_space='MNI152',
                                            trg_space='fsaverage',
                                            resampling='transform_to_alt',
                                            alt_spec=('fsaverage', '10k'))
print(nsynth, genepc)

###############################################################################
# Once the images are resampled we can easily correlate them:

from neuromaps import stats
corr = stats.compare_images(nsynth, genepc)
print(f'Correlation: r = {corr:.02f}')

###############################################################################
# What if we want to assess the statistical significance of this correlation?
# In this case, we can use a null model from the :mod:`neuromaps.nulls` module.
#
# Here, we'll employ the null model proposed in Alexander-Bloch et al., 2018,
# *NeuroImage*. We provide one of the maps we're comparing, the space + density
# of the map, and the number of permutations we want to generate. The returned
# array will have two dimensions, where each row corresponds to a vertex and
# each column to a unique permutation.
#
# (Note that we need to pass the loaded data from the provided map to the null
# function so we use the :func:`neuromaps.images.load_data` utility.)

from neuromaps import images, nulls
nsynth_data = images.load_data(nsynth)
rotated = nulls.alexander_bloch(nsynth_data, atlas='fsaverage', density='10k',
                                n_perm=100, seed=1234)
print(rotated.shape)

###############################################################################
# We can supply the generated null array to the
# :func:`neuromaps.stats.compare_images` function and it will be used to
# generate a non-parameteric p-value. The function assumes that the array
# provided to the `nulls` parameter corresponds to the *first* dataset passed
# to the function (i.e., `nsynth`).
#
# Note that the correlation remains identical to that above but the p-value is
# now returned as well:

corr, pval = stats.compare_images(nsynth, genepc, nulls=rotated)
print(f'Correlation: r = {corr:.02f}, p = {pval:.04f}')

###############################################################################
# There are a number of different null functions that can be used to generate
# null maps; they have (nearly) identical function signatures, so refer to the
# :ref:`API reference <ref_nulls>` for more information.

Neurosynth:  /home/tamires/neuromaps-data/annotations/neurosynth/cogpc1/MNI152/source-neurosynth_desc-cogpc1_space-MNI152_res-2mm_feature.nii.gz
Gene PC1:  ['/home/tamires/neuromaps-data/annotations/abagen/genepc1/fsaverage/source-abagen_desc-genepc1_space-fsaverage_den-10k_hemi-L_feature.func.gii', '/home/tamires/neuromaps-data/annotations/abagen/genepc1/fsaverage/source-abagen_desc-genepc1_space-fsaverage_den-10k_hemi-R_feature.func.gii']
(<nibabel.gifti.gifti.GiftiImage object at 0x2ad003d2cb60>, <nibabel.gifti.gifti.GiftiImage object at 0x2ad003d2da90>) (<nibabel.gifti.gifti.GiftiImage object at 0x2ad08891fc80>, <nibabel.gifti.gifti.GiftiImage object at 0x2ad08891e540>)


TypeError: argument should be a str or an os.PathLike object where __fspath__ returns a str, not 'GiftiImage'